In [1]:
import json, pandas as pd

In [2]:
experiments = [ experiment for experiment in json.load(open('../experiments/wd_gpt-4_serialization_from_wikidata_results.json', 'r')) if "confusion_matrix" in experiment ]

In [3]:
def experiment_df(experiment):
    df = pd.DataFrame.from_records(experiment['data'])
    df["class_id"] = experiment['concept']['id']
    df["class_label"] = experiment['concept']['label']
    df["class_serialization"] = experiment['concept']['serialization']
    return df[["class_id", "class_label", "class_serialization", "id", "label", "serialization", "actual", "predicted", "rationale"]]

In [4]:
df = pd.concat([ experiment_df(experiment) for experiment in experiments ], ignore_index=True)
df_fp = df[((df["actual"] == 'negative') & (df["predicted"] == 'positive'))].reset_index()
df_fn = df[((df["actual"] == 'positive') & (df["predicted"] == 'negative'))].reset_index()

In [5]:
df_fp.to_csv('wikidata_gpt-4_serialization_FPs.csv')
df_fn.to_csv('wikidata_gpt-4_serialization_FNs.csv')

In [6]:
len(df_fp)

22

In [7]:
len(df_fn)

120

In [8]:
df_fp_reviewed = pd.read_csv(open('wikidata_gpt-4_serialization_FPs_reviewed.csv', 'r'))
df_fn_reviewed = pd.read_csv(open('wikidata_gpt-4_serialization_FNs_reviewed.csv', 'r'))
df_reviewed = pd.concat([df_fp_reviewed, df_fn_reviewed], ignore_index=True)

In [9]:
len(df_reviewed)

142

In [10]:
df_reviewed.head(3)

,class_id,class_label,class_serialization,id,label,serialization,actual,predicted,rationale,review,rationale sound
0,http://www.wikidata.org/entity/Q30324006,History of cities,<http://www.wikidata.org/entity/Q30324006>\t<h...,http://www.wikidata.org/entity/Q1025460,Caesaromagus,<http://www.wikidata.org/entity/Q1025460>\t<ht...,negative,positive,"Caesaromagus, as a Roman settlement in Beauvai...",LM in error,no
1,http://www.wikidata.org/entity/Q30324006,History of cities,<http://www.wikidata.org/entity/Q30324006>\t<h...,http://www.wikidata.org/entity/Q2914723,Nemausus,<http://www.wikidata.org/entity/Q2914723>\t<ht...,negative,positive,"The concept ""History of cities"" refers to the ...",LM in error,no
2,http://www.wikidata.org/entity/Q30324006,History of cities,<http://www.wikidata.org/entity/Q30324006>\t<h...,http://www.wikidata.org/entity/Q2932881,Caesarodunum,<http://www.wikidata.org/entity/Q2932881>\t<ht...,negative,positive,"Caesarodunum, as an entity, could be considere...",LM in error,no


In [11]:
len(df_reviewed[(df_reviewed['actual'] == 'positive') & 
                (df_reviewed['predicted'] == 'negative')])

120

In [12]:
len(df_reviewed[(df_reviewed['review'] == 'KG in error') & 
                (df_reviewed['actual'] == 'positive') & 
                (df_reviewed['predicted'] == 'negative')])

38

In [13]:
len(df_reviewed[(df_reviewed['actual'] == 'negative') & 
                (df_reviewed['predicted'] == 'positive')])

22

In [14]:
len(df_reviewed[(df_reviewed['review'] == 'KG in error') & 
                (df_reviewed['actual'] == 'negative') & 
                (df_reviewed['predicted'] == 'positive')])

13